<a href="https://colab.research.google.com/github/kurota0612/test/blob/main/GitTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 引き当てたGPUの情報を確認する
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



##各種設定

In [ ]:
import os

# Google Cloud SDKの権限許可
try:
    from google.colab import auth
    auth.authenticate_user()
except ImportError:
    pass

# GitHubのリポジトリ名
REPOSITORY_NAME = 'kaggleops-tutorial'
# GCEのプロジェクト名
PROJECT_ID = 'kaggleops-project'
# GCSのバケット名
BUCKET_NAME = 'kaggleops-bucket'

# 環境変数を設定してる
# 左辺は辞書。"GCLOUD_PROJECT"はkey。そのvalueとして「PROJECT_ID」
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [ ]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
# Colabのローカルにリポジトリをclone
!git clone https://github.com/kurota0612/{REPOSITORY_NAME}.git

Cloning into 'kaggleops-tutorial'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 69 (delta 22), reused 59 (delta 12), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [ ]:
# パッケージの一括インストール
!pip install -qr ./{REPOSITORY_NAME}/requirements.txt

In [ ]:
# 現在のディレクトリを変更
cd ./{REPOSITORY_NAME}/src

In [ ]:
# Google Cloundの設定
!gcloud config set project {PROJECT_ID}

In [ ]:
# !gsutil -mq cp -rn gs://{BUCKET_NAME}/mlruns /content/{BUCKET_NAME}/src

# 
blobs = storage_client.list_blobs(BUCKET_NAME)
for blob in blobs:
    filepath = blob.name
    if filepath[:7]=="mlruns/" and len(filepath)!=7:
        os.makedirs("/".join(filepath.split("/")[:-1]), exist_ok=True)
        blob.download_to_filename(filepath)

##プログラム実行

mlproject.pyの中でMLprojectファイル（YAML？）を実行してる

⇨MLprojectファイルの中でtrain.pyを実行してる

⇨train.pyが実際の学習コード

In [ ]:
!python mlproject.py

##ローカルに保存されたmlrunsをGCSに転送

ローカル（Colab？）に保存されたmlflowの結果ファイルをGCSに転送する

⇨後でGCSからGCEに結果ファイルをコピーしてブラウザから結果を確認する

In [ ]:
# !gsutil -mq cp -rn /content/{REPOSITORY_NAME}/src/mlruns gs://{BUCKET_NAME}

import glob

files = [f for f in glob.glob("mlruns/**", recursive=True) if f[:7]=="mlruns/" and len(f)!=7]
files_in_bucket = [f.name for f in storage_client.list_blobs(BUCKET_NAME)]
bucket = storage_client.get_bucket(BUCKET_NAME)

for f in files:
    try:
        if f not in files_in_bucket:
            blob = bucket.blob(f)
            blob.upload_from_filename(f)
    except:
        pass